<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
موضوع‌بندی
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک تمرین صنعتی از  کاربرد یادگیری ماشین در پردازش زبان طبیعی (NLP) خوش آمدید.  در این تمرین داده‌های واقعی وب فارسی که توسط پلتفرم <a href="https://www.yektanet.com/">یکتانت</a> پالایش و جمع‌آوری شده در اختیار ما قرار گرفته است. هدف تمرین؛ ساخت یک مدل یادگیری ماشین است که با توجه متن‌های موجود در یک پیوند (Link) نظیر <i>عنوان</i>، <i>توضیحات</i>، <i>محتوای متنی [کامل]</i> و غیره بتواند دسته‌ی موضوعی آن سند را پیش‌بینی کند. به‌عنوان مثال اگر پیوندی از یک سایت خبری با عنوان «<i>کیهان کلهر جایزه مرد سال موسیقی جهان را دریافت کرد</i>» داشته باشیم، مدل ما باید پیش‌بینی کند که این مطلب مرتبط با موضوع «موسیقی» است. البته در این مثال ما تنها از ویژگی <i>عنوان</i> یاد کردیم، در حالی‌که می‌توان از متن <i>توضیحات</i> یا <i>محتوای متنی</i> هم کمک گرفت. 

</br>
توجه داشته باشید برای آن‌که بتوانید از الگوریتم‌های معرفی‌شده در کالج جهت کار روی داده‌های متنی استفاده کنید نیاز است حداقل با یکی از روش‌های تعبیه‌سازی (Embedding) آشنا شده باشید تا بتوانید هر متن را به یک بردار ویژگی عددی تبدیل کنید.

</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [1]:
import numpy as np
import pandas as pd
from hazm import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    هر نمونه از این مجموعه‌داده با ویژگی‌هایی که در جدول زیر شرح داده شده همراه است. ستون <code>category</code> متغیر هدف مسئله است که موضوع محتوا را نشان می‌دهد. 
</font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>| موضوع (متغیر هدف) |
|<code>description</code>| توضیحات |
|<code>text_content</code>| محتوای متنی |
|<code>title</code>| عنوان |
|<code>h1</code>| محتوای تگ <code>h1</code> صفحه |
|<code>h2</code>|محتوای تگ <code>h2</code> صفحه  |
|<code>url</code>| آدرس پیوند|
|<code>domain</code>|دامنه‌ی وب‌سایت |
|<code>id</code>|آیدی پیوند|

</font>
</div>
</center>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>yektanet_train.csv</code> و نمونه‌های آزمون که باید موضوع آن‌ها را پیش‌بینی کنید در فایل <code>yektanet_test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از مجموعه‌ی آموزشی را به عنوان مجموعه‌ی اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [2]:
train = pd.read_csv('../data/yektanet_train.csv') # To-Do
# train.head()

In [3]:
test = pd.read_csv('../data/yektanet_test.csv') # To-Do
# test.head()

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام کار با داده‌های متنی، پیش‌پردازش داده‌ها به کمک تکنیک‌های پردازش زبان طبیعی یکی از مراحل بسیار تاثیرگذار در یادگیری مدل و عملکرد نهایی است.
در تمرین «کامنت‌کاوی» فصل دسته‌بندی با چندین تکنیک رایج پیش‌پردازش خصوصاً برای زبان فارسی آشنا شدید. در این قسمت نیز می‌توانید تابعی بنویسید که یک رشته (<code>string</code>) را گرفته، اصلاحات موردنظر شما رو روی متن اعمال کرده و در نهایت نتیجه را با فرمت یک رشته (<code>string</code>) خروجی دهد. بررسی و تحلیل کلمات موجود در مجموعه‌داده از نظر تعداد رخداد نیز می‌تواند شما را در پیش‌پردازش بهتر یاری کند.
</font>
</p>


In [4]:
class PreprocessingKoosha:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def fill_na_train(self):
        cols = ['description', 'h1', 'h2', 'domain']

        for col in cols:
            most_frequent = self.train[col].value_counts().sort_values(ascending=False).index[0]
            self.train[col].fillna(most_frequent, inplace=True)
            self.test[col].fillna(most_frequent, inplace=True)
        self.train.isna().sum()

    def encoding_label(self):    
        encoder = LabelEncoder()
        self.train['category'] = encoder.fit_transform(self.train['category'])     

    def define_train_data_target(self):
        self.train_data_target = self.train['category']
        self.train.drop(columns='category', axis=1, inplace=True)

    def text_embeding(self):
        # Difine a tokenizer with hazm library
        stopwords = stopwords_list()
        punctuations = ['؟', '!', '.', ',', ';', ':', '،', '؛', '(', ')', '[', ']', '{', '}', '«', '»', '|', '/']
        normalizer = Normalizer()
        tokenizer = WordTokenizer()
        def hazm_tokenizer(text):
            
            # Normalize the text
            text_normalized = normalizer.normalize(text)
            tokens = tokenizer.tokenize(text_normalized)
            # tokens = [token for token in tokens if token not in punctuations and token not in stopwords and not token.isdigit()]
            tokens = [token for token in tokens if token not in punctuations and token not in stopwords]
            return tokens
        
        # Define a pipeline using countvectorize and tfid
        pipe = Pipeline([('count', CountVectorizer(tokenizer=hazm_tokenizer, analyzer='word', ngram_range=(1, 2),
                                         min_df=5, lowercase=False)),
                        ('tfidf', TfidfTransformer(sublinear_tf=True))])

        pipe.fit(self.train['title'])
        self.train_data = pipe.transform(self.train['title']).toarray()
        self.test_data = pipe.transform(self.test['title']).toarray()


    def balance_target(self):
        ros = RandomOverSampler()
        self.train_data, self.train_data_target = ros.fit_resample(self.train_data, self.train_data_target)

    def define_validation_data(self):
        self.train_data, self.valid_data, self.train_data_target, self.valid_data_target = train_test_split(
            self.train_data, self.train_data_target, test_size=.1)
        
    def transform(self):
        self.fill_na_train()
        self.encoding_label()
        self.define_train_data_target()
        self.text_embeding()
        self.balance_target()
        self.define_validation_data()

        return (self.train_data, self.train_data_target,
                self.valid_data, self.valid_data_target,
                self.test_data)

In [5]:
preprocessor = PreprocessingKoosha(train=train, test=test)
# train_data, test_data = preprocessor.transform()
train_data, train_data_target, valid_data, valid_data_target, test_data = preprocessor.transform()

c:\Users\BLUENOTEBOOK\Desktop\media\ex\qenv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
valid_data.shape

(1351, 1879)

In [7]:
train_data.shape

(12157, 1879)

In [8]:
test_data.shape

(417, 1879)

In [9]:
# To-Do

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام تعبیه‌سازی به کمک کتابخانه‌ی <code>scikit-learn</code>
از آنجا که ممکن است توکن‌ساز (Tokenizer) مورد استفاده در این کتابخانه مناسب زبان فارسی نباشد، بهتر است از توکن‌سازهای مناسب این زبان استفاده کنیم. کافیست تابعی بنویسید که یک رشته (<code>string</code>) را گرفته و به کمک کتابخانه‌ی موردنظر شما (مثل <code>word_tokenize</code> در کتابخانه‌ی هضم) توکن‌های آن را جدا کند. خروجی تابع لیستی از توکن‌ها خواهد بود.
</font>
</p>


In [10]:
# def tokenizer(text):
#     return None # To-Do

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید انتخاب این‌که از کدام ویژگی‌ها (متن‌ها) به‌عنوان ورودی الگوریتم استفاده کنید بر عهده‌ی خودتان است. می‌توانید تنها از یک ستون مثل <code>title</code> استفاده کنید یا می‌توانید به‌نحوی متن یا حتی بردار ویژگی هر ستون را با همدیگر ترکیب کنید. همچنین فراموش نکنید که ستون متغیر هدف یعنی <code>category</code> نیاز به کدگذاری دارد.
</font>
</p>


In [11]:
# from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# train['category'] = encoder.fit_transform(train['category'])
# To-Do

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون وقت آن رسیده که الگوریتم یادگیری ماشین موردنظر خود را بر روی داده‌های آموزشی اجرا کنید. در انتخاب الگوریتم کاملاً آزاد هستید. برای این بخش می‌توانید ابتدا هر ورودی متن را به کمک تکنیک‌های معرفی‌شده در درسنامه‌های این فصل (مثل Bag-of-Word یا Tf-idf) به بردارهای ویژگی عددی تبدیل کنید. سپس این بردارها را همراه با لیست برچسب‌های صحیح به الگوریتم بدهید تا مدل آموخته شود. علاوه بر این می‌توانید تمام این مراحل را در یک خط لوله‌ (<code>Pipeline</code>) نیز تعریف کنید.
</font>
</p>

In [12]:
from sklearn.svm import LinearSVC

model = LinearSVC(loss='hinge', penalty='l2', max_iter=5000)
model.fit(train_data, train_data_target)

LinearSVC(loss='hinge', max_iter=5000)

In [13]:
# To-Do

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>F1-score</code> نام دارد و از رویکرد میانگین‌گیری وزن‌دار (<code dir=ltr>average='weighted'</code>) استفاده می‌شود.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید و طبق نتایج به‌دست‌آمده پارامترهای مدل خود را بهتر تنظیم کنید.
</font>
</p>

In [14]:
from sklearn.metrics import f1_score

pred_valid = model.predict(valid_data)
f1_score(pred_valid, valid_data_target, average='weighted')
# To-Do

0.8790416149149453

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدل‌سازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در مجموعه‌ی آزمون استفاده کنید و نتایج را در یک دیتافریم تک‌ستونه با نام <code>submission</code> و در قالب زیر آماده کنید. توجه داشته باشید که ترتیب پیش‌بینی شما اهمیت دارد یعنی به عنوان مثال پیش‌بینی مدل برای نمونه‌ی آزمون <code>m</code> ام را باید در ردیف <code>m</code> ام این دیتافریم ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>|پیش‌بینی مدل شما (از جنس رشته)|
    
</font>
</div>
</center>

In [15]:
pred_test = model.predict(test_data)

submission = pd.DataFrame() # To-Do
submission['category'] = pred_test
submission.replace({0: 'اجتماعی', 1: 'اشتغال', 2: 'تجارت و اقتصاد', 3: 'تحصیلات', 4: 'تکنولوژی و کامپبوتر', 5: 'حقوق و دولت و سیاست', 6: 'حیوانات خانگی', 7: 'خانه و باغبانی', 8: 'خانواده', 9: 'خودرو', 10: 'سفر و گردشگری', 11: 'سلامت', 12: 'علم و دانش', 13: 'غذا و نوشیدنی', 14: 'فیلم و سینما', 15: 'مد و زیبایی', 16: 'مذهبی', 17: 'مسکن', 18: 'موسیقی', 19: 'هنر و سرگرمی', 20: 'ورزش', 21: 'کتاب و ادبیات'}, inplace=True)

submission.head()

,category
0,هنر و سرگرمی
1,حقوق و دولت و سیاست
2,حقوق و دولت و سیاست
3,موسیقی
4,خانواده


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [16]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['text_categorization.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['text_categorization.ipynb', 'submission.csv']


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
💭 اضافه: ابرِ کلمات (Word Cloud)
</font>
</h2>

<center>
<img src="wordcloud.png">
</center>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
یکی از کتابخانه‌های بسیار جالب مرتبط با متن در پایتون، <a href="https://github.com/amueller/word_cloud"><code>WordCloud</code></a> نام دارد. این کتابخانه به شما کمک می‌کند تا ابری از پرتکرارترین توکن‌های موجود در یک مجموعه‌متن را به شکلی زیبا به تصویر بکشید. خوشبختانه نسخه‌ی فارسی این کتابخانه نیز وجود دارد که می‌توانید از <a href="https://github.com/alihoseiny/word_cloud_fa">این لینک</a> به صفحه‌ی گیت‌هاب آن مراجعه کنید. حتی می‌توانید به‌صورت دلخواه تصویری را تعیین کنید تا نمایش نهایی توکن‌ها تداعی‌کننده‌ی تصویر موردنظر شما باشد.
</font>
</p>

In [18]:
cloud_text = ''
for text in train_data:
    cloud_text += text + ' '

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None

In [ ]:
from wordcloud_fa import WordCloudFa

wc = WordCloudFa(width=1200, height=800, persian_normalize=True, include_numbers=False, max_words=120, background_color='white', min_font_size=10, max_font_size=100)
word_cloud = wc.generate(cloud_text)
image = word_cloud.to_image()
image.show()
image.save('wordcloud.png')

<h4 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>راهنمایی</b>
</font>
</h4>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ۱. از تکنیک n-gram کمک بگیرید.
    <br>
    ۲. توازن مجموعه‌داده را بررسی کنید.
    <br>
    ۳. در پیش‌پردازش خود می‌توانید حذف حروف اضافه و اعداد، حذف کلمات توقف، نرمال‌سازی و... را آزمایش کنید.
</font>
</p>